In [1]:
import numpy as np
import mido
import json
import os
from mido import Message, MidiFile, MidiTrack, MetaMessage
import glob
from os import listdir
from os.path import isdir, isfile, join

In [2]:
'''Functions'''
def midifile_to_dict(mid): #將midi轉為dict
    tracks = []
    for track in mid.tracks:
        tracks.append([vars(msg).copy() for msg in track])

    return {
        'ticks_per_beat': mid.ticks_per_beat,
        'tracks': tracks,
    }

def dict_to_midifile(midi_dict, save_file_path): #將dict轉回midi
    #midi_dict必須要是track 0 是metadata, track 1是events
    outfile = MidiFile()
    meta_track = MidiTrack()
    event_track = MidiTrack()
    outfile.tracks.append(meta_track)
    outfile.tracks.append(event_track)
    meta_track.append(MetaMessage('track_name', name = midi_dict['tracks'][0][0]['name']))
    meta_track.append(MetaMessage('set_tempo', tempo = midi_dict['tracks'][0][1]['tempo']))
    meta_track.append(MetaMessage('time_signature', numerator = midi_dict['tracks'][0][2]['numerator'], \
                      denominator = midi_dict['tracks'][0][2]['denominator'], clocks_per_click = midi_dict['tracks'][0][2]['clocks_per_click'], \
                      notated_32nd_notes_per_beat = midi_dict['tracks'][0][2]['notated_32nd_notes_per_beat']))
    event_track.append(MetaMessage('track_name', name = midi_dict['tracks'][1][0]['name']))
    for idx, event in enumerate(midi_dict['tracks'][1]):
        if event['type'] == 'note_on' or event['type'] == 'note_off':
            event_track.append(Message(event['type'], note = event['note'], time = event['time'], velocity = event['velocity']))
        elif event['type'] == 'pitchwheel':
            event_track.append(Message(event['type'], pitch = event['pitch'], time = event['time']))
        elif event['type'] == 'control_change':
            event_track.append(Message(event['type'], control = event['control'], value = event['value'], time = event['time']))
    outfile.save(save_file_path)

def read_lyrics_phon(song_data_path, song_name): #讀取已標記好的phonetic的檔案，並且標上其note在phrase的位置
    with open(os.path.join(song_data_path, song_name, song_name + '.phon'), 'rb') as file:
        phon_txt = file.read()
    with open(os.path.join(song_data_path, song_name, song_name + '.lyrics'), 'rb') as file:
        lyrics_txt = file.read()
    phon_phrase = phon_txt.decode('utf8').replace(' ','').split('\r\n') #轉成utf-8，再去掉空白，最後用\r\n來坐唱句切割成list
    phon_phrase[0] = phon_phrase[0].replace('\ufeff', '') #刪掉BOM
    lyrics_phrase = lyrics_txt.decode('utf8').replace(' ','').split('\r\n') #轉成utf-8，再去掉空白，最後用\r\n來坐唱句切割成list
    lyrics_phrase[0] = lyrics_phrase[0].replace('\ufeff', '') #刪掉BOM
    phon = []
    lyrics = []
    for i in phon_phrase:
        phon.append(i.split(','))
    for i in lyrics_phrase:
        lyrics.append(i.split(','))
    lyrics_phon_with_locate = []
    for idx_phrase, phrase in enumerate(phon):
        find_start = False
        for idx, j in enumerate(phrase):
            if j != 'bre' and find_start == False:
                find_start = True
                lyrics_phon_with_locate.append([lyrics[idx_phrase][idx], j, 'phrase_start'])
            elif j == 'bre':
                lyrics_phon_with_locate.append([lyrics[idx_phrase][idx], j, 'breath'])
            elif idx == len(phrase) - 1:
                lyrics_phon_with_locate.append([lyrics[idx_phrase][idx], j, 'phrase_end'])
            elif find_start == True :
                lyrics_phon_with_locate.append([lyrics[idx_phrase][idx], j, 'phrase_middle'])        
    return lyrics_phon_with_locate

def convert_to_events_arrays(note_events): #將event取出各類別的資料
    note_events_abs_time = []
    for event in note_events:
        note_events_abs_time.append(event['time'])
    for idx, event in enumerate(note_events):
        if idx != 0:
            note_events_abs_time[idx] = note_events_abs_time[idx-1] + note_events_abs_time[idx]
    #將note event存成note on 和note off的兩個2d-list，[[timestamp 1, pitch 1],....]
    note_on = []
    note_off = []
    PIT = []
    DYN = []
    PBS = 8 #預設為8
    for idx, event in enumerate(note_events):
        if event['type'] == 'note_on':
            note_on.append([note_events_abs_time[idx],event['note']])
        elif event['type'] == 'note_off':
            note_off.append([note_events_abs_time[idx],event['note']])
        elif event['type'] == 'pitchwheel':
            PIT.append([note_events_abs_time[idx],event['pitch']])
        elif event['type'] == 'control_change' and event['control'] == 11: #前面的等號應該可以刪掉(?)
            DYN.append([note_events_abs_time[idx],event['value']])
        elif event['type'] == 'control_change' and event['control'] == 6: #讀取PBS值，Variaudio是放在cc6
            PBS = event['value']
   
    return note_on, note_off, PIT, DYN, PBS

def combine_events(note_on, note_off, PIT, DYN): #合併所有事件變成note_all_events
    PIT = np.array(PIT)
    note_all_events = []
    for note_idx, note_on_event in enumerate(note_on):
        find_start = False
        for pit_idx, pitch_event in enumerate(PIT):
            if pitch_event[0] >= note_on_event[0] and find_start == False:
                pit_start_idx = pit_idx
                find_start = True
            if pitch_event[0] > note_off[note_idx][0] and find_start ==True: #PIT的值是對note頭不對尾
                pit_end_idx = pit_idx - 1
                break
            elif pit_idx == len(PIT) - 1:
                pit_end_idx = pit_idx
                break
        note_all_events.append([note_on_event[1], note_on_event[0], note_off[note_idx][0], pit_start_idx, pit_end_idx])
    #note_all_events[音高, 開始時間, 結束時間, PIT開始idx, PIT結束idx]
    return note_all_events

def fix_PIT(note_all_events, note_on_ref, PIT_ori_local, PBS):
    PIT_ori_local = np.array(PIT_ori_local)
    for idx, note in enumerate(note_all_events):    
        if note[0] != note_on_ref[idx][1]:
            shift_pitch = (note_on_ref[idx][1] - note[0]) * 8192/PBS
            for pit_idx, pitch in enumerate(PIT_ori_local[:,1][note[3]:note[4]+1]):
                fix_pitch = pitch - shift_pitch
                if fix_pitch > 8191:
                    fix_pitch = 8191
                elif fix_pitch < -8192:
                    fix_pitch = -8192
                PIT_ori_local[:,1][note[3] + pit_idx] = fix_pitch
            note_all_events[idx][0] = note_on_ref[idx][1]
    return PIT_ori_local.tolist()

def interpolate_limit_PIT(note_all_events, PIT_ori_local):
    PIT_ori_local = np.array(PIT_ori_local)
    PIT_ori_local = np.insert(PIT_ori_local, 2, 0, axis=1)
    for idx, note in enumerate(note_all_events):
        for pit_idx, pitch in enumerate(PIT_ori_local[:,1][note[3]:note[4]+1]):
            if pitch > 8000 or pitch < -8000:
                if pit_idx == len(PIT_ori_local[:,1][note[3]:note[4]+1]) - 1:
                    PIT_ori_local[:,1][note[3] + pit_idx] = PIT_ori_local[:,1][note[3] + pit_idx - 1]
                    PIT_ori_local[:,2][note[3] + pit_idx] = 1
                elif pit_idx == 0:
                    PIT_ori_local[:,1][note[3] + pit_idx] = PIT_ori_local[:,1][note[3] + pit_idx + 1]
                    PIT_ori_local[:,2][note[3] + pit_idx] = 1
                else:
                    time_ref = PIT_ori_local[:,0][note[3] + pit_idx]
                    first_time = PIT_ori_local[:,0][note[3] + pit_idx - 1]
                    second_time = PIT_ori_local[:,0][note[3] + pit_idx + 1]
                    fisrt_PIT = PIT_ori_local[:,1][note[3] + pit_idx - 1]
                    second_PIT = PIT_ori_local[:,1][note[3] + pit_idx + 1]
                    fix_pitch = (fisrt_PIT*(time_ref - second_time) + second_PIT*(first_time - time_ref))/(first_time - second_time)
                    PIT_ori_local[:,1][note[3] + pit_idx] = fix_pitch
                    PIT_ori_local[:,2][note[3] + pit_idx] = 1
    
    return PIT_ori_local.tolist()
                    
def fix_midi(note_events, note_all_events, PIT_fix):
    PIT_fix = np.array(PIT_fix)
    note_on_idx = 0
    note_off_idx = 0
    PIT_idx = 0
    for idx, event in enumerate(note_events):
        if event['type'] == 'note_on':
            note_events[idx]['note'] = note_all_events[note_on_idx][0]
            note_on_idx += 1
        elif event['type'] == 'note_off':
            note_events[idx]['note'] = note_all_events[note_off_idx][0]
            note_off_idx += 1
        elif event['type'] == 'pitchwheel':
            note_events[idx]['pitch'] = PIT_fix[:,1][PIT_idx]
            PIT_idx += 1
    return note_events

###目前尚未用到###
def separate_breath(note_all_events):
    #去除呼吸的部分, breath的音高是24
    note_all_events_debre = []
    breath_events = []
    for note_event in note_all_events:
        if note_event[0] == 24:
            breath_events.append(note_event)
        else:
            note_all_events_debre.append(note_event)
    return note_all_events_debre, breath_events

In [3]:
'''Infomation
This program is to fix the PIT message by referenced the reference note.

After that, it converts the midi files to the json file which include all of the information.

The song directory need to be placed at 'data_path/song' with the song_name.

The json file will be saved in 'data_path/json', and the midi file will be saved in 'data_path/midi'

with the same name as the song_name.
'''

'''Usage
Args: song_name, data_path

Input: original midi(A.mid), reference note midi(B.mid), score midi(C.mid)

Output: all information json, PIT fixed midi

'''

'''輸入歌曲名稱和data的資料夾'''

def midi_lyrics_to_json(song_name):
    data_path = '../data'

    #設定各種路徑
    song_data_path = os.path.join(data_path, 'song')
    json_save_path = os.path.join(data_path, 'json')
    midi_save_path = os.path.join(data_path, 'midi')

    #定義三個midi檔的位置->轉成mido的midifile->轉成dict
    ori_midi = midifile_to_dict(mido.MidiFile(os.path.join(song_data_path, song_name, 'A.mid')))


    #取出三個midi檔的note_on, note_off, PIT, DYN
    note_on_ori, note_off_ori, PIT_ori, DYN_ori, PBS = convert_to_events_arrays(ori_midi['tracks'][1])

    #將note的資訊合併
    note_all_events = combine_events(note_on_ori, note_off_ori, PIT_ori, DYN_ori)
    ###note_all_events[音高, 開始時間, 結束時間, PIT開始idx, PIT結束idx]###

    #The tempo is microseconds per beat, that is bpm = 60/(temp*10^(-6))
    tempo = 60/(ori_midi['tracks'][0][1]['tempo']*10**(-6))
    ticks_per_beat = (ori_midi['ticks_per_beat'])

    if save_check == True:
        #儲存成json檔
        print('儲存' + song_name +'的json檔')
        midi_events_dict = {'song_name':song_name, 'tempo':tempo, 'ticks_per_beat':ticks_per_beat, 'note_all_events':note_all_events, \
                            }
        with open(os.path.join(json_save_path, song_name + '.json'), 'w') as outfile:
            json.dump(midi_events_dict, outfile)
        with open(os.path.join(json_save_path, song_name + '_ori.json'), 'w') as outfile:
            json.dump(ori_midi, outfile)

        #儲存成midi檔
        print('儲存' + song_name +'的midi檔')
        midi_fix_track = fix_midi(ori_midi['tracks'][1], note_all_events, PIT_fix)
        midi_fix_dict = ori_midi
        midi_fix_dict['tracks'][1] = midi_fix_track
        dict_to_midifile(midi_fix_dict, os.path.join(midi_save_path, song_name + '.mid'))

In [6]:
data_path = 'midi_data'

song_list = [f for f in listdir(song_data_path) if isdir(join(song_data_path, f))]
json_exist = [f.replace('.json', '') for f in listdir(json_save_path) if isfile(join(json_save_path, f))]

for song_name in song_list:
    if song_name not in json_exist:
        print(song_name)
        midi_lyrics_to_json(song_name)

21033_chorus
21033_chorus的note和phonetic數量一致
儲存21033_chorus的json檔
儲存21033_chorus的midi檔
